In [1]:
# Day_02_04_tf_layers.py
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import  input_data

In [2]:
def mnist_tf_layers():
    mnist = input_data.read_data_sets('mnist')

    ph_x = tf.placeholder(tf.float32)
    ph_y = tf.placeholder(tf.int32)
    # ph_keep = tf.placeholder(tf.float32)
    ph_training = tf.placeholder(tf.bool)

    x_image = tf.reshape(ph_x, [-1, 28, 28, 1])

    conv_1 = tf.layers.conv2d(x_image, 32, [3, 3],
                              activation=tf.nn.relu, padding='SAME',
                              kernel_initializer=tf.random_normal_initializer())

    pool_l = tf.layers.max_pooling2d(conv_1, [2, 2], [2, 2], padding='SAME')

    ################################################################################33
    conv_2 = tf.layers.conv2d(pool_l, 64, [3, 3],
                              activation=tf.nn.relu, padding='SAME',
                              kernel_initializer=tf.random_normal_initializer())

    pool_2 = tf.layers.max_pooling2d(conv_2, [2, 2], [2, 2], padding='SAME')

    flat = tf.layers.flatten(pool_2)

    full_3 = tf.layers.dense(flat, 256, activation=tf.nn.relu)
    drop_3 = tf.layers.dropout(full_3, 0.5, training=ph_training)

    z = tf.layers.dense(drop_3, 10)
    hx = tf.nn.softmax(z)

    # return

    # ---------------------------------------------------------------------------------------------------------#

    # tensorflow 2.0d에서 정리 될것,  logit=예측  label을 결과괌
    # logit에 z를 하는것은 hx를 전달하지 않아도 됨, 자체적으로 함
    # 우리의 결과는 ont-hot 백터가 아님으로  sparse_softmax_cross_entropy_with_logits
    loss_i = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=z, labels=ph_y)
    loss = tf.reduce_mean(loss_i)

    # 모델 정확도 높이기 위해 아님  ->  빨리 수렴한다.
    # optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train = optimizer.minimize(loss=loss)

    # sess로 변수의 값을 알수 있다.
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    epochs = 1
    batch_size = 100   # 출구를 몇개씩 조사할건지
    n_iters = mnist.train.num_examples // batch_size # 550

    # epcochs를 1번 돌고 념
    for i in range(epochs):
        c = 0
        for j in range(n_iters):
            xx, yy = mnist.train.next_batch(batch_size)

            sess.run(train, feed_dict={ph_x: xx, ph_y: yy, ph_training: True})
            c += sess.run(loss, {ph_x: xx, ph_y:yy, ph_training: True})

        print(i, c/ n_iters)

    preds = sess.run(hx, {ph_x: mnist.test.images, ph_training: False})
    preds_arg = np.argmax(preds, axis=1)  # 1: 수평, 0: 수직
    # spase일때는 argmax를 가져올필요 없다.
    # test_arg  = np.argmax(mnist.test.labels, axis=1)

    # 파이썬의 list는 broadcasting기능이 없어서, numpy array로 변경
    # grades = np.array(['Setosa', 'Versicolor', 'Virginica'])
    # print(grades[preds_arg])
    # print(preds)

    # 1차 혼돈 : 데이터가 섞여 있지 않음으로 인한오류 → shuffle 필요 np.random.shuffle(iris)
    # 2차 혼돈 : 돌릴때 마다 위치가 달라져서 ...np.random.seed(1)
    print('acc: ', np.mean(preds_arg == mnist.test.labels))

    print(preds_arg)
    # print(test_arg)

    sess.close()

In [ ]:
mnist_tf_layers()